### SQL BASIC
#### What is SQL?
SQL (Structured Query Language) is a language used to manage data in relational databases.

Think of it like Excel formulas but for databases – it lets you create, read, update, and delete data in a structured way.

Documentation
Link to: sqlite3

Core SQL Commands
Here are the four essential SQL commands (often called CRUD operations):

### SELECT – Read data

In [ ]:
SELECT * FROM coins; -- Gets all data from coins table

SELECT name, price FROM coins WHERE price > 100; — Gets specific columns with a condition

: 

### INSERT NEW RECORDS

In [ ]:
INSERT INTO coins VALUES ('Bitcoin', 50000); — Adds a new row

INSERT INTO coins (name, price) VALUES ('Ethereum', 3000); — Adds to specific columns

#### UPDATE – Modify Existing Records

In [ ]:
UPDATE coins SET price = 51000 WHERE name = 'Bitcoin'; 
-- Changes Bitcoin's price


#### DELETE – Remove Records

In [ ]:
DELETE FROM coins WHERE price < 10; 
-- Deletes all cheap coins


### Setting Up a Simple SQLite3 Database in Python
This tutorial creates a simple database to track cryptocurrencies.

Install pandas (if needed)

In [ ]:
pip install pandas


: 

In [ ]:
pip install pandas
